In [1]:
from gurobipy import GRB
import gurobipy as gp
import os
import torch
import time
import sys
sys.path.append('/home/peilun/ml4uc_experiment')
from model.gcn import GNNPolicy
from dataset import GraphDataset
import torch_geometric
from utilities import *
import numpy as np

In [2]:
# GAP = 1e-3
# for i in range(20):
#     with gp.Env(empty=True) as env:
#         env.setParam('OutputFlag', 0)
#         env.setParam('MIPGap', GAP)
#         env.start()
#         env.setParam('Threads', 1)
#         m = gp.read(f'50_0_1_w/3bin/test/1/{i}_0.lp',env)
#         begin = time.time()
#         m.Params.outputflag = 0
#         m.optimize()
#         m.Params.Threads = 1
#         print(time.time() - begin)

In [3]:
instance = '50_0_1_w'
month=7
data_path_1bin = f'{instance}/3bin/test/{month+1}'
files_1bin = os.listdir(data_path_1bin)
files_1bin = [f for f in files_1bin if f.endswith('.lp')]
print(files_1bin)
N=int(instance.split('_')[0])
T=24
k=0.95
# data_path_3bin = f'{instance}_opt/3bin/test'
# files_3bin = os.listdir(data_path_3bin)

['212_0.lp', '212_3.lp', '212_4.lp', '212_1.lp', '213_3.lp', '212_2.lp', '213_0.lp', '213_1.lp', '215_0.lp', '215_2.lp', '215_3.lp', '213_2.lp', '215_4.lp', '213_4.lp', '214_4.lp', '214_0.lp', '214_1.lp', '214_2.lp', '214_3.lp', '216_2.lp', '216_3.lp', '215_1.lp', '216_4.lp', '216_0.lp', '217_0.lp', '216_1.lp', '217_2.lp', '217_1.lp', '217_3.lp', '217_4.lp', '219_2.lp', '219_0.lp', '220_3.lp', '220_2.lp', '218_0.lp', '220_1.lp', '218_1.lp', '218_2.lp', '221_0.lp', '218_3.lp', '218_4.lp', '219_1.lp', '221_1.lp', '219_4.lp', '219_3.lp', '221_4.lp', '220_0.lp', '222_1.lp', '220_4.lp', '222_2.lp', '221_2.lp', '223_0.lp', '221_3.lp', '222_4.lp', '223_2.lp', '223_4.lp', '222_0.lp', '224_2.lp', '224_0.lp', '223_1.lp', '222_3.lp', '225_3.lp', '226_1.lp', '223_3.lp', '224_1.lp', '226_2.lp', '224_4.lp', '224_3.lp', '225_0.lp', '227_0.lp', '225_1.lp', '225_2.lp', '225_4.lp', '227_1.lp', '227_2.lp', '226_0.lp', '226_3.lp', '226_4.lp', '227_4.lp', '227_3.lp', '228_0.lp', '228_2.lp', '228_3.lp', '22

In [4]:
# model_save_path_1bin = f'../model_save/3bin_50_0_1_w_1/model_best.pth'
model_save_path_3bin = f'../model_save/3bin_10_std/model_best.pth'

# model_1bin = GNNPolicy()
# state_1bin_dict = torch.load(model_save_path_1bin, map_location=torch.device('cpu'))
# model_1bin.load_state_dict(state_1bin_dict)
# model_1bin.eval()

model_3bin = GNNPolicy()
state_3bin_dict = torch.load(model_save_path_3bin, map_location=torch.device('cpu'))
model_3bin.load_state_dict(state_3bin_dict)
model_3bin.eval()

GNNPolicy(
  (cons_embedding): Sequential(
    (0): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=4, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
  )
  (edge_embedding): Sequential(
    (0): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
  )
  (var_embedding): Sequential(
    (0): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=6, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
  )
  (conv_v_to_c): BipartiteGraphConvolution()
  (conv_c_to_v): BipartiteGraphConvolution()
  (conv_v_to_c2): BipartiteGraphConvolution()
  (conv_c_to_v2): BipartiteGraphConvolution()
  (output_module): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=False)
  )
)

In [5]:
# valid_data_1bin = GraphDataset(f'{instance}/1bin/train/2/')
# valid_iter_1bin = torch_geometric.loader.DataLoader(
#     valid_data_1bin, batch_size = 64, shuffle = False,
#     num_workers = 2)
valid_data_3bin = GraphDataset('../datasets/10_std/3bin/test/')
valid_iter_3bin = torch_geometric.loader.DataLoader(
    valid_data_3bin, batch_size = 64, shuffle = False,
    num_workers = 2)


In [6]:
# valid_loss_1bin = train(model_1bin, valid_iter_1bin, None)
# print(valid_loss_1bin)
#0.03

In [7]:
# k1=0.9
# k0=0.02
GAP = 1e-3
local_search_range=0
TLE=200

In [8]:
# def local_search(m,sol):
#     instance_variabels = m.getVars()
#     ans = [0.5 for i in range(N*T)]
#     A_indices, A_values, v_nodes, c_nodes, b_vars=get_fea(m)
#     k1=0.9
#     k0=0.1
#     p0,p1=0,0


#     for i in range(N*T):
#         if sol[i]>=k1 or sol[i]<=k0:
#             x_star = 1 if sol[i]>=k1 else 0
#             if x_star==0:
#                 p0+=1
#             else:
#                 p1+=1
#             ans[i]=x_star

#     # for i,v in enumerate(instance_variabels):
#     #     if b_vars[i]==1:
#     #         if sol[i]>=k1 or sol[i]<=k0:
#     #             x_star = 1 if sol[i]>=k1 else 0
#     #             if x_star==0:
#     #                 p0+=1
#     #             else:
#     #                 p1+=1
#     #             ans[i]=x_star
#                 # m.addConstr(v == x_star, name=f'alpha_up_{v.VarName}')
    
#     return ans

In [9]:
# def fix_variable(m,ans):
#     instance_variabels = m.getVars()[:len(ans)]
#     p0,p1 = 0,0
#     for i in range(len(ans)):
#         if ans[i]==0 or ans[i]==1:
#             if ans[i]==0:
#                 p0+=1
#             else:
#                 p1+=1
#             m.addConstr(instance_variabels[i] == ans[i], name=f'alpha_{instance_variabels[i].VarName}')
#     m.update()
#     print('p0:',p0,'p1:',p1)

#     return m

In [10]:
def get_pmax_pmin_spin(path,m):
    spin=[0 for i in range(24)]
    constrs = m.getConstrs()
    for i in constrs:
        if 'System_spinning_reserve_requirement' in i.ConstrName:
            #System_spinning_reserve_requirement后面中括号里的数字表示下标
            spin[int(i.ConstrName.split('[')[1].split(']')[0])]=-m.getAttr("RHS", [i])[0]
    uc = UC(path)
    pmax = uc.ThPimax
    pmin = uc.ThPimin
    # spin = uc.Spin
    return pmax,pmin,spin

In [11]:
def he_fs(m,sol,instance):
    pmax,spin = get_pmax_spin(m,N,T)
    if os.path.exists(f'./UC_AF/{instance}.mod'):
        pmax,pmin,spin = get_pmax_pmin_spin(f'./UC_AF/{instance}.mod',m)
    else:
        pmax,pmin,spin = get_pmax_pmin_spin(f'../UC_AF/{instance}.mod',m)
    kmin=[]
    kmax=[]
    for t in range(T):
        p_list=[]
        for i,idx in enumerate(range(t,N*T,T)):
            p_list.append((sol[idx],pmax[i],idx,pmin[i]))
        #从大到小排序
        p_list.sort(key=lambda x:x[0], reverse=True)
        #从大到小排序
        p_list.sort(key=lambda x:x[0], reverse=True)
        sumpmax=0
        sumpmin=0
        k1=0
        kt=0
        for i in range(len(p_list)):
            sumpmax+=p_list[i][1]
            if k1==0 and sumpmax>=spin[t]:
                k1=i+1
            sumpmin+=p_list[i][3]
            if kt==0 and sumpmin>=spin[t]:
                kt=i+1

        kmin.append(k1)
        kmax.append(kt)
        
    return kmin,kmax

In [12]:
def ps_accelerate(file):
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.setParam('MIPGap', GAP)
        env.start()
        m = gp.read(file,env)

        A_indices, A_values, v_nodes, c_nodes, b_vars=get_fea(m)

        sol = model_3bin(c_nodes, A_indices, A_values, v_nodes).sigmoid()
        #只保留b_vars=1的位置的变量

        sol = sol[b_vars==1]
        kmin,kmax = he_fs(m,sol,instance)

        m.optimize()

        #获得u变量的解
        u = m.getVars()[:N*T]
        ux = np.around(m.getAttr("X", u)).astype(int).reshape(-1,24)
        #对列求和
        ux = np.sum(ux,axis=0)
        print('============================')
        print(kmin)
        print(ux)


In [13]:
# name = '50_0_1_w'
# u0_save=np.load(f'../instances/fixed/{name}/u0_save.npy')
# p0_save=np.load(f'../instances/fixed/{name}/p0_save.npy')
# onoff_save=np.load(f'../instances/fixed/{name}/onoff_save.npy')

In [14]:
# day=222
# n=31
# print(p0_save[day][n],u0_save[day][n],onoff_save[day][n])

In [15]:
cnt1=0
cnt0=0
cnts=0
import numpy as np
for file in files_1bin[:10]:
    path = os.path.join(data_path_1bin,file)
    ps_accelerate(path)
    # if t==1:
    #     cnt1+=1
    # elif t==2:
    #     cnts+=1
    # else:
    #     cnt0+=1
    # print(cnt0,cnts,cnt1)


[31, 21, 25, 27, 23, 20, 21, 20, 22, 24, 24, 23, 22, 24, 24, 26, 28, 26, 23, 21, 21, 21, 19, 18]
[30 30 30 28 28 28 29 30 31 31 30 30 29 28 28 28 30 32 33 33 32 31 30 30]
[31, 21, 26, 27, 23, 21, 21, 20, 22, 24, 24, 24, 23, 24, 25, 27, 28, 27, 23, 21, 21, 21, 19, 18]
[31 31 30 28 28 28 29 30 31 31 31 31 30 30 29 30 30 33 34 34 33 32 31 31]
[31, 21, 26, 27, 23, 20, 21, 20, 22, 24, 24, 23, 23, 24, 24, 26, 28, 26, 23, 21, 20, 21, 19, 18]
[30 30 30 28 28 28 29 30 31 31 30 30 29 29 29 29 30 32 33 33 32 31 30 30]
[32, 21, 26, 28, 23, 21, 21, 20, 22, 25, 24, 24, 23, 24, 25, 27, 28, 27, 23, 21, 21, 21, 19, 19]
[31 31 30 28 28 28 29 29 31 31 31 30 29 29 29 29 30 32 33 33 32 31 30 30]
[30, 21, 25, 25, 22, 21, 20, 19, 21, 23, 22, 23, 23, 23, 24, 26, 25, 24, 22, 20, 20, 21, 19, 18]
[30 30 30 29 28 28 28 28 30 30 30 29 29 28 28 28 30 31 32 32 31 31 30 29]
[31, 21, 25, 27, 23, 20, 21, 20, 22, 24, 24, 23, 22, 24, 24, 26, 28, 26, 23, 21, 20, 21, 19, 18]
[30 30 30 28 28 28 29 30 31 31 30 30 29 28 28 28

In [16]:
# c = m.getConstrByName('alpha_u[32,13]')
# print(m.getAttr("RHS", [c])[0])

In [17]:
# cnt1=0
# cnt0=0
# import numpy as np
# for file in files_1bin:
#     path = os.path.join(data_path_1bin,file)
#     # t = ps_accelerate(path)
#     # if t==1:
#     #     cnt1+=1
#     # else:
#     #     cnt0+=1
#     # print(cnt0,cnt1)



#     m = ps_accelerate(path)
#     # uv = m.getVars()[:50*24]
#     # # pv = m.getVars()[50*24:100*24]
#     # u = np.array([v.X for v in uv]).reshape(50,24)
#     # print(u[33][0])
#     # # p = np.array([v.X for v in pv]).reshape(50,24)
#     # # np.savetxt('u.csv',u,delimiter=',')
#     # # np.savetxt('p.csv',p,delimiter=',')
#     # m.write('temp.lp')
#     if m.Status!=2 and m.Status!=9:
#         print(file)
#         m.computeIIS()
#         for c in m.getConstrs():
#             if c.IISConstr:
#                 print(c.constrName)
#     # tot=0
#     # for i in range(len(ans)):
#     #     if i%24==0:
#     #         print('')
#     #     print(ans[i],end=' ')
#     # print('')
#     # print(tot)
#     # # # print(len(ans))
#     # ans = np.array(ans).reshape(50,24)
#     # #将ans存入excel
#     # np.savetxt('a.csv',ans,delimiter=',')
